<a href="https://colab.research.google.com/github/cooolbabu/GoogleGemini101/blob/main/AzureDatabricks/Boiler_code_for_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install PyGithub

In [ ]:
# @title Github Helper functions
repository = "cooolbabu/GoogleGemini101" # @param ["cooolbabu/GoogleGemini101"]


In [2]:
#pip install PyGithub
from google.colab import userdata
from github import Github

# Get the OpenAI API key from Colab secrets
github_token=userdata.get('Github_Token')
# Initialize a GitHub instance
g = Github(github_token)

def read_file_as_string(file_path):
  """
    Reads the file and return a string representation of the file contents

    Parameters:
      file_path (str): Filename including filepath
  """
  try:
      with open(file_path, 'r') as file:
          file_contents = file.read()
      return file_contents
  except FileNotFoundError:
      print(f"File '{file_path}' not found.")
      return None
  except Exception as e:
      print(f"An error occurred: {e}")
      return None

def check_in_file(repo_name, file_path, file_content, content_tag, branch):
  """
    Checks if a specific file exists in a GitHub repository and updates it with new content if it does.
    If the file does not exist, it creates a new file with the provided content.

    This function operates on a specific branch named 'test'. If updating, it will commit the changes with a given content tag as the commit message.
    In case the file needs to be created, it will also use the content tag as the commit message for the new file.

    Parameters:
    - repo_name (str): The name of the repository, formatted as 'username/repository'.
    - file_path (str): The path to the file within the repository. This should include the file name and its extension.
    - file_content (str): The content to be written to the file. This is used both for updating and creating the file.
    - content_tag (str): A message associated with the commit used for updating or creating the file.
    - branch (str): Github branch for the code

    Behavior:
    - If the file exists at the specified path, it updates the file with `file_content`, using `content_tag` as the commit message.
    - If the file does not exist, it creates a new file at the specified path with `file_content`, also using `content_tag` as the commit message for creation.
    - Upon successful update or creation, prints a success message indicating the action taken.
 """

  # Get the repository
  repo = g.get_repo(repo_name)

  try:
      # Get the contents of the file if it exists
      file = repo.get_contents(file_path, ref=branch)

      # Update the file
      repo.update_file(file_path, content_tag, file_content, file.sha, branch=branch)
      print(f"File '{file_path}' updated successfully.")
  except:
      # If the file doesn't exist, create it
      print(f"{file_path}/{file_content} does not exist")
      repo.create_file(file_path, content_tag, file_content, branch=branch)
      print(f"File '{file_path}' created successfully.")


In [ ]:
# @title Github functions

file_contents = read_file_as_string('/content/export_007.json')
check_in_file(repository, "AzureDatabricks/Test_file", file_contents, "Initial checkin of export_007_json", branch="test")
